In [41]:
import ROOT
import numpy as np
#from root_numpy import random_sample
from scipy.ndimage import uniform_filter
import time
import matplotlib.pyplot as plt
import multiprocessing as mp

<h1>Simulation Model</h1>


In [45]:
# the constants of the constants. Pretty much choose once and leave it.
C_total_channels = int(3.6 * 1e6)
C_channels_per_module = int(15*30)
C_total_modules = int(C_total_channels/C_channels_per_module)

# skip
#C_expected_interactions_per_BC = 10
#C_expected_number_of_particles_reaching_the_detector_per_interaction = 10
#C_max_radius = 10

### Skipping radius dependency for now...



def get_number_above_threshold(threshold, count, list_):
    """counts the number of elements in the list above the threshold"""
    arr = np.fromiter(list_, dtype=np.float, count=count)
    above_count = (arr > threshold).sum()
    return above_count

def sample_random_signals(prng, signal_mu, signal_sigma, signal_count):
    """Samples randomly from the Landau distrubution"""
    return [ prng.Landau(signal_mu, signal_sigma) for _ in range(signal_count) ]

def sample_random_noise(prng, noise_sigma, noise_count):
    """Samples randomly from the Gaus distrubution"""
    return [ prng.Gaus(0, noise_sigma) for _ in range(noise_count) ]

def get_number_of_readouts(ratio_of_channels_to_sample):
    """Returns the number of channels to read from"""
    return int(C_total_channels * ratio_of_channels_to_sample)

def simulation_get_counts(prng, number_of_BC, ratio_of_channels_to_sample, signal_to_total_event_ratio, threshold, signal_mu, signal_sigma, noise_sigma):
    """Runs a simulation over specified number of BC, and returns two lists: signal and noise counts above threshold for each BC"""
    # simulated
    result_signal_count = np.zeros(number_of_BC)
    result_noise_count = np.zeros(number_of_BC)
    
    readouts = get_number_of_readouts(ratio_of_channels_to_sample)
    signal_count_per_BC = int(round(readouts * signal_to_total_event_ratio))
    noise_count_per_BC = readouts - signal_count_per_BC
    
    for BC in range(number_of_BC):    
        signal_samples = sample_random_signals(prng, signal_mu, signal_sigma, signal_count_per_BC)
        noise_samples = sample_random_noise(prng, noise_sigma, noise_count_per_BC)
        
        signal_above_count = get_number_above_threshold(threshold, signal_count_per_BC, signal_samples)
        noise_above_count = get_number_above_threshold(threshold, noise_count_per_BC, noise_samples)
        
        result_signal_count[BC] = int(signal_above_count)
        result_noise_count[BC] = int(noise_above_count)
    
    #print(result_noise_count)
    return np.asarray([result_signal_count, result_noise_count])

def simulation_noise_count_only():
    """Simulation of the large time frame, but it contains signal as well, I don't know..."""
    pass

def simulation_combined_get_counts(number_of_BC, ratio_of_channels_to_sample, signal_to_total_event_ratio, threshold, signal_mu, signal_sigma, noise_sigma):
    """Runs the simulations in parallel inorder to faster simulate. It then combines each result.
        and returns two lists: signal and noise counts above threshold for each BC"""
    # profiles to determine compute resources
    profiles = {"normal": 2, "fast": 4, "fastest": 99}
    max_processes = profiles["fastest"]  # edit this
    
    # setup pool
    nprocs = min(mp.cpu_count(), max_processes)
    pool = mp.Pool(processes=nprocs)
    
    # simulate
    seed = 123
    bins_of_BC = eq_div(number_of_BC, nprocs)
    param_list = [(ROOT.TRandom3(seed + i), int(bins_of_BC[i]), ratio_of_channels_to_sample, signal_to_total_event_ratio, threshold, signal_mu, signal_sigma, noise_sigma) \
                     for i in range(nprocs)]
    
    results = np.asarray(pool.starmap(simulation_get_counts, param_list))
    results = np.hstack(results).squeeze() # reshape
    pool.close()
    pool.join()
    
    signal_count_array = results[0]
    noise_count_array = results[1]
    
    # list of count,  count for each bunch crossing event
    return signal_count_array, noise_count_array

# helper methods
def eq_div(N, i):
    return [N // i + 1] * (N % i) + [N // i] * (i - N % i)
def np_stats(np_list):
    """Use a list from e.g. simulaiton_combined_get_counts and get statistics about that list"""
    return np.mean(np_list), np.var(np_list), np.std(np_list)
def print_count_statistics(np_list):
    """Use a list from e.g. simulaiton_combined_get_counts and get statistics about that list"""
    mean, variance, standard_deviation = np_stats(np_list)
    print("mean:", mean)
    print("variance:", variance)
    print("standard_deviation:", standard_deviation)



def get_deviation_from_expected_list_count(list_signal, list_noise, number_of_BC, ratio_of_channels_to_sample, signal_to_total_event_ratio, threshold, signal_mu, signal_sigma, noise_sigma):
    """Get the deviation from the simulated signals vs the expected signals.
        Returns both the total deviation and deviation for each BC"""
    low = -100
    high = 500
    
    # expected ratios
    fNoise = ROOT.TF1("ElecNoiseBg", "TMath::Gaus(x, 0, %f)" % noise_sigma, low, high)
    fNoise.SetNpx(1000)
    fSignal = ROOT.TF1("Signal", "TMath::Landau(x, %f, %f)" % (signal_mu, signal_sigma), low, high)
    fSignal.SetNpx(1000)

    expected_noise_above_to_total_noise_ratio = 1 - fNoise.Integral(low, threshold)/fNoise.Integral(low, high)
    expected_signal_above_to_total_signal_ratio = 1 - fSignal.Integral(low, threshold)/fSignal.Integral(low, high)
    
    # calculate how many channels where used, and signal, noise for channels
    readouts = get_number_of_readouts(ratio_of_channels_to_sample)
    signal_count_per_BC = int(round(readouts * signal_to_total_event_ratio))
    noise_count_per_BC = readouts - signal_count_per_BC
    total_signal_count = signal_count_per_BC * number_of_BC
    total_noise_count = noise_count_per_BC * number_of_BC
    
    event_ratio = signal_to_total_event_ratio
    scale_threshold_to_total = 1 / (event_ratio*expected_noise_above_to_total_noise_ratio + (1-event_ratio)*expected_signal_above_to_total_signal_ratio)

    # list_signal, list_noise contains the true number of events for each bunch crossing
    
    # this is a value we get from the detector
    simulated_total_above_count = list_signal.sum() + list_noise.sum()
    computed_total_count = simulated_total_above_count * scale_threshold_to_total
    computed_signal_count = computed_total_count * event_ratio
    total_deviation = abs(1 - computed_signal_count/total_signal_count)
    
    # this is a value we get from the detector
    simulated_above_count_for_BC = list_signal + list_noise
    computed_total_count_for_BC = simulated_above_count_for_BC * scale_threshold_to_total
    computed_signal_count_for_BC = computed_total_count_for_BC * event_ratio
    deviation_list = np.abs(1 - computed_signal_count_for_BC / signal_count_per_BC)
    #deviation_list = computed_signal_count_for_BC - signal_count_per_BC
    
    #print(list_noise.sum())
    #print((list_signal.sum()/expected_signal_above_to_total_signal_ratio)/total_signal_count, \
    #      (list_noise.sum()/expected_noise_above_to_total_noise_ratio)/total_noise_count)
    
    # return both the total combined, and the deviation for each BC
    return total_deviation, deviation_list



def main():
    ratio_of_channels_to_sample = 1.0  # used to downscale the simulation (larger error though)
    
    # There is a bug here, only works in multiples of 16. Should take ~1.5 seconds per BC at 1.0 simulation
    number_of_BC = 16*4   # one second is 1*40e6
    
    signal_to_total_event_ratio = 0.4/4
    #signal_to_total_event_ratio = 0.5
    
    threshold = 40
    signal_mu = 100 # mV
    reference_singal_mu = 100 # from s.15
    signal_sigma = 0.1 + (25-0.1) * (signal_mu/reference_singal_mu) #mv  25 from reference for signal_mu = 100 (s.24)
    noise_sigma = 5 #mV
    
    lists = simulation_combined_get_counts(
                        number_of_BC,
                        ratio_of_channels_to_sample,
                        signal_to_total_event_ratio,
                        threshold,
                        signal_mu,
                        signal_sigma,
                        noise_sigma)
    
    """dev = get_deviation_from_expected_list_count(
                        lists[0],
                        lists[1],
                        number_of_BC,
                        ratio_of_channels_to_sample,
                        signal_to_total_event_ratio,
                        threshold,
                        signal_mu,
                        signal_sigma,
                        noise_sigma)"""
    print(1)


if __name__ == '__main__':
    mp.freeze_support()
    t0 = time.time()
    main()
    t1 = time.time()
    print("Simulation took:", round(t1-t0, 1), "seconds")

    
# TODO: there is a bug with a bias, look below all values are negative

1
Simulation took: 77.3 seconds


====================================================


<h1>Testing</h1>


Number of CPU cores to use: 8
C


Process ForkPoolWorker-23:
Process ForkPoolWorker-19:
Process ForkPoolWorker-20:
Process ForkPoolWorker-17:
Process ForkPoolWorker-24:
Process ForkPoolWorker-18:
Process ForkPoolWorker-22:
Process ForkPoolWorker-21:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib64/python3.6/multiprocessing/process.py", line 258, in _bo

KeyboardInterrupt: 

  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "<ipython-input-5-a5b1eef0e377>", line 116, in calculate_error_array
    , dtype=np.float, count=steps)
  File "/usr/lib64/python3.6/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(

  File "<ipython-input-5-a5b1eef0e377>", line 43, in error_of_expected_signal
    noise_iter = [ prng.Gaus(0, noise_sigma) for _ in range(noise_count) ]
  File "<ipython-input-5-a5b1eef0e377>", line 43, in <listcomp>
    noise_iter = [ prng.Gaus(0, noise_sigma) for _ in range(noise_count) ]
  File "<ipython-input-5-a5b1eef0e377>", line 43, in error_of_expected_signal
    noise_iter = [ prng.Gaus(0, noise_sigma) for _ in range(noise_count) ]
KeyboardInterrupt
  File "<ipython-input-5-a5b1eef0e377>", line 43, in error_of_expected_signal
    noise_iter = [ prng.Gaus(0, noise_sigma) for _ in range(noise_count) ]
  File "<ipython-input-5-a5b1eef0e377>", line 43, in error_of_expected_signal
    noise_iter = [ prng.Gaus(0, noise_sigma) for _ in range(noise_count) ]
  File "<ipython-input-5-a5b1eef0e377>", line 43, in <listcomp>
    noise_iter = [ prng.Gaus(0, noise_sigma) for _ in range(noise_count) ]
KeyboardInterrupt
  File "<ipython-input-5-a5b1eef0e377>", line 43, in <listcomp>
    noise_